This is the third assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Just execute all cells one after the other and you are done - just note that in the last one you must update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"


In [0]:
credentials_1 = {'password':"""*****""",
                 'custom_url':'*****',
                 'username':'*****' }



Let's create a SparkSession object and put the Cloudant credentials into it

In [0]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .getOrCreate()

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [0]:
df=spark.read.load('shake_classification', "org.apache.bahir.cloudant")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+--------+-----+-----+-----+--------------------+--------------------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+--------+-----+-----+-----+--------------------+--------------------+
|    1|aaaaaaaa|-0.01|-0.03|-0.09|3492765e6ac38cd24...|1-2c827a2b55caa1e...|
|    1|aaaaaaaa|-0.01|  0.0| -0.1|3492765e6ac38cd24...|1-86ceff170907fc5...|
|    1|aaaaaaaa| 0.01|  0.0|-0.07|3492765e6ac38cd24...|1-22ba2f3ccff5cde...|
|    1|aaaaaaaa|-0.01|  0.0|-0.08|3492765e6ac38cd24...|1-978a5470abb20cb...|
|    1|aaaaaaaa| 0.01|  0.0|-0.06|3492765e6ac38cd24...|1-96f8550e65d9c9f...|
|    1|aaaaaaaa|  0.0|  0.0|-0.09|3492765e6ac38cd24...|1-1f5ca0558b17a16...|
|    1|aaaaaaaa| 0.01|  0.0|-0.07|3492765e6ac38cd24...|1-22ba2f3ccff5cde...|
|    1|aaaaaaaa|  0.0|  0.0|-0.07|3492765e6ac38cd24...|1-8fe62ca7a975b42...|
|    1|aaaaaaaa|  0.0|  0.0|-0.07|3492765e6ac38cd24...|1-8fe62ca7a975b42...|
|    1|aaaaaaaa|-0.01|  0.0|-0.08|3492765e6ac38cd24...|1-978a5470abb20cb...|

Let’s check if we have balanced classes – this means that we have roughly the same number of examples for each class we want to predict. This is important for classification but also helpful for clustering

In [0]:
spark.sql("SELECT count(class), class from df group by class").show()

+------------+-----+
|count(class)|class|
+------------+-----+
|         813|    0|
|         708|    1|
+------------+-----+



Let's create a VectorAssembler which consumes columns X, Y and Z and produces a column “features”


In [0]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X","Y","Z"],
                                  outputCol="features")

Please insatiate a clustering algorithm from the SparkML package and assign it to the clust variable. Here we don’t need to take care of the “CLASS” column since we are in unsupervised learning mode – so let’s pretend to not even have the “CLASS” column for now – but it will become very handy later in assessing the clustering performance. PLEASE NOTE – IN REAL-WORLD SCENARIOS THERE IS NO CLASS COLUMN – THEREFORE YOU CAN’T ASSESS CLASSIFICATION PERFORMANCE USING THIS COLUMN 



In [0]:
from pyspark.ml.clustering import KMeans

clust = KMeans().setK(12).setSeed(6)

Let’s train...


In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, clust])
model = pipeline.fit(df)

...and evaluate...

In [0]:
prediction = model.transform(df)
prediction.show()

+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|           features|prediction|
+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|    1|aaaaaaaa|-0.01|-0.03|-0.09|3492765e6ac38cd24...|1-2c827a2b55caa1e...|[-0.01,-0.03,-0.09]|         0|
|    1|aaaaaaaa|-0.01|  0.0| -0.1|3492765e6ac38cd24...|1-86ceff170907fc5...|   [-0.01,0.0,-0.1]|         0|
|    1|aaaaaaaa| 0.01|  0.0|-0.07|3492765e6ac38cd24...|1-22ba2f3ccff5cde...|   [0.01,0.0,-0.07]|         0|
|    1|aaaaaaaa|-0.01|  0.0|-0.08|3492765e6ac38cd24...|1-978a5470abb20cb...|  [-0.01,0.0,-0.08]|         0|
|    1|aaaaaaaa| 0.01|  0.0|-0.06|3492765e6ac38cd24...|1-96f8550e65d9c9f...|   [0.01,0.0,-0.06]|         0|
|    1|aaaaaaaa|  0.0|  0.0|-0.09|3492765e6ac38cd24...|1-1f5ca0558b17a16...|    [0.0,0.0,-0.09]|         0|
|    1|aaaaaaaa| 0.01|  0.0|

In [0]:
prediction.createOrReplaceTempView('prediction')
spark.sql('''
select max(correct)/max(total) as accuracy from (

    select sum(correct) as correct, count(correct) as total from (
        select case when class != prediction then 1 else 0 end as correct from prediction 
    ) 
    
    union
    
    select sum(correct) as correct, count(correct) as total from (
        select case when class = prediction then 1 else 0 end as correct from prediction 
    ) 
)
''').rdd.map(lambda row: row.accuracy).collect()[0]

0.6896778435239974

If you reached at least 55% of accuracy you are fine to submit your predictions to the grader. Otherwise please experiment with parameters setting to your clustering algorithm, use a different algorithm or just re-record your data and try to obtain. In case you are stuck, please use the Coursera Discussion Forum. Please note again – in a real-world scenario there is no way in doing this – since there is no class label in your data. Please have a look at this further reading on clustering performance evaluation https://en.wikipedia.org/wiki/Cluster_analysis#Evaluation_and_assessment


In [0]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2019-02-11 01:55:13--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2019-02-11 01:55:14 (17.5 MB/s) - ‘rklib.py’ saved [2289/2289]



In [0]:
prediction= prediction.repartition(1)
prediction.write.json('a2_m3.json')

In [0]:
!zip -r a2_m3.json.zip a2_m3.json

  adding: a2_m3.json/ (stored 0%)
  adding: a2_m3.json/_SUCCESS (stored 0%)
  adding: a2_m3.json/._SUCCESS.crc (stored 0%)
  adding: a2_m3.json/.part-00000-3c80bfc8-1134-43f9-b78b-e3de36985dd6.json.crc (stored 0%)
  adding: a2_m3.json/part-00000-3c80bfc8-1134-43f9-b78b-e3de36985dd6.json (deflated 84%)


In [0]:
!base64 a2_m3.json.zip > a2_m3.json.zip.base64

In [0]:
from rklib import submit
key = "*****"
part = "****"
email = "annmargaret.tutu@icloud.com"
secret = "*****"

with open('a2_m3.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"Cu6KW","id":"f_F-qCtuEei_fRLwaVDk3g~Cu6KW~bXlRmC3SEemkyBLJ0_dFYg","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
